In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cleaned-realestate/cleaned_realestate.csv


In [2]:
import pandas as pd 
from sklearn.preprocessing import MinMaxScaler

In [3]:
InputDir="/kaggle/input/cleaned-realestate/cleaned_realestate.csv"
df=pd.read_csv(InputDir)

In [4]:
df.head()

,price,level,levels,rooms,area,kitchen_area,geo_lat,geo_lon,building_type,object_type,postal_code,street_id,id_region,house_id
0,10700000,4,13,3,85.0,12.0,55.540060,37.725112,3,0,142701,242543,50,681306
1,2500000,2,3,1,30.0,9.0,44.738685,37.713668,3,2,353960,439378,23,1730985
2,1450000,5,5,2,47.0,6.0,48.511172,44.566846,2,0,400096,260588,34,1009994
3,9000000,2,4,3,107.4,21.3,55.009914,82.934859,4,0,630102,233285,54,2823596
4,2300000,16,18,1,39.7,11.5,45.003869,39.086511,4,0,350065,523822,23,1284243


In [5]:
df = df.sample(frac=0.004, replace=True)


In [6]:
df = df.reset_index(drop=True)
df['id'] = df.reset_index().index

In [7]:
df.head()

,price,level,levels,rooms,area,kitchen_area,geo_lat,geo_lon,building_type,object_type,postal_code,street_id,id_region,house_id,id
0,10700000,2,24,3,75.0,25.0,47.223378,39.748931,0,0,344019,543555,61,1096542,0
1,2799000,5,9,1,29.7,6.0,54.960748,82.971476,2,0,630066,548809,54,2124101,1
2,1750000,1,2,0,33.0,12.0,44.145521,43.099081,0,0,357242,273787,26,702316,2
3,5850000,11,27,1,42.1,12.7,60.081004,30.340985,3,0,194358,535271,78,1047907,3
4,3400000,1,5,2,43.0,6.0,55.163114,37.430224,2,0,142304,176856,50,2680486,4


In [8]:
df.shape

(15245, 15)

# **entity**

**property实体**

In [9]:
df_property=df[['id','price','level','levels','rooms','area','kitchen_area','house_id']].copy()

**house实体**

In [10]:
df_house = df[['house_id', 'geo_lat', 'geo_lon','street_id']].copy()
df_house.drop_duplicates(subset='house_id', inplace=True)

**street实体** 

In [11]:
df_street = df[['street_id','postal_code','id_region']].copy()
df_street.drop_duplicates(subset='street_id', inplace=True)

**region实体**

In [12]:
df_region = df[['id_region']].copy()
df_region.drop_duplicates(subset='id_region', inplace=True)

In [13]:
df['avg_geo_lat'] = df.groupby('id_region')['geo_lat'].transform('mean')
df['avg_geo_lon'] = df.groupby('id_region')['geo_lon'].transform('mean')

In [14]:
df_region=df[['id_region','avg_geo_lat','avg_geo_lon']].copy()
df_region.drop_duplicates(subset='id_region', inplace=True)
df_region.shape

(77, 3)

In [15]:
from geopy.distance import geodesic 

In [16]:
lat=df_region['avg_geo_lat'].mean()
lon=df_region['avg_geo_lon'].mean()
dis=[]
for index,line in df_region.iterrows():
    coor1 = (line.iloc[1], line.iloc[2])
    coor2=(lat,lon)
    distance = geodesic(coor1, coor2).meters
    dis.append(distance)
average = sum(dis) / len(dis)
average

1501991.8559150638

In [17]:
df_property.to_csv("/kaggle/working/property.csv")
df_house.to_csv("/kaggle/working/house.csv")
df_street.to_csv("/kaggle/working/street.csv")
df_region.to_csv("/kaggle/working/region.csv")